# Mobile Get Input Notebook - Phase 2

**Phase 2 Improvements**: Functions moved to modular `profile_extraction` module for reusability.

This notebook automates creation of terrain profiles for ITU-R P.1812-6 propagation prediction.

## Workflow
1. **Setup**: Configure transmitter location and parameters
2. **Generate Receivers**: Create uniformly distributed receiver points
3. **Extract Profiles**: Get terrain elevation and land cover data (via module)
4. **Export**: Save profiles to CSV for batch processing

## Imports

In [ ]:
import os
import math
import sys
from pathlib import Path

import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Point
from dataclasses import dataclass
from typing import Iterable, Union

# Find and add project root to path
# Check multiple possible locations for src/ and config_sentinel_hub.py
possible_roots = [
    Path.cwd(),                              # Current directory
    Path.cwd().parent,                       # Parent directory
    Path.cwd().parent.parent,                # Grandparent (from notebooks subdir)
    Path('/Users/oz/Documents/mst_gis'),   # Absolute path
]

project_root = None
for root in possible_roots:
    if (root / 'src').exists() or (root / 'config_sentinel_hub.py').exists():
        project_root = root
        break

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Import profile extraction module
from mst_gis.propagation.profile_extraction import (
    landcover_at_point,
    resolve_credentials,
    generate_profile_points,
)

# Import Sentinel Hub config
from config_sentinel_hub import (
    SH_CLIENT_ID, SH_CLIENT_SECRET,
    TOKEN_URL, PROCESS_URL, COLLECTION_ID,
)

## Path Setup

In [ ]:
# Detect project root and create data directories
notebook_dir = Path.cwd()
project_root = Path.cwd() if (Path.cwd() / 'data').exists() else Path.cwd().parent

profiles_dir = project_root / 'data' / 'input' / 'profiles'
api_data_dir = project_root / 'data' / 'intermediate' / 'api_data'
workflow_dir = project_root / 'data' / 'intermediate' / 'workflow'

profiles_dir.mkdir(parents=True, exist_ok=True)
api_data_dir.mkdir(parents=True, exist_ok=True)
workflow_dir.mkdir(parents=True, exist_ok=True)

print(f"Project root: {project_root}")
print(f"Profiles dir: {profiles_dir}")

## Configuration

In [ ]:
# ============================================================================
# CONFIGURATION - Edit these parameters for your simulation
# ============================================================================

CONFIG = {
    'TRANSMITTER': {
        'tx_id': 'TX_0001',
        'longitude': -13.40694,
        'latitude': 9.345,
        'antenna_height_tx': 57,
        'antenna_height_rx': 10,
    },
    'P1812': {
        'frequency_ghz': 0.9,
        'time_percentage': 50,
        'polarization': 1,
    },
    'RECEIVER_GENERATION': {
        'max_distance_km': 11,
        'azimuth_step': 10,
        'distance_step': 0.5,
        'sampling_resolution': 30,
    },
    'SENTINEL_HUB': {
        'buffer_m': 11000,
        'chip_px': 734,
        'year': 2020,
    },
    'LCM10_TO_CT': {
        100: 1, 80: 2, 30: 2, 40: 2, 70: 2, 110: 2, 254: 2,
        20: 3, 50: 3, 10: 4, 60: 4, 90: 4,
    },
    'CT_TO_R': {1: 0, 2: 0, 3: 10, 4: 15, 5: 20},
}

# Derived values
tx_lon = CONFIG['TRANSMITTER']['longitude']
tx_lat = CONFIG['TRANSMITTER']['latitude']
max_distance_km = CONFIG['RECEIVER_GENERATION']['max_distance_km']
n_points = int(max_distance_km * 1000 / CONFIG['RECEIVER_GENERATION']['sampling_resolution'])
azimuths = list(range(0, 360, CONFIG['RECEIVER_GENERATION']['azimuth_step']))
distances = np.arange(
    CONFIG['RECEIVER_GENERATION']['distance_step'],
    max_distance_km + CONFIG['RECEIVER_GENERATION']['distance_step'],
    CONFIG['RECEIVER_GENERATION']['distance_step']
)

print(f"Transmitter: ({tx_lat}, {tx_lon})")
print(f"Azimuths: {len(azimuths)} | Profile points: {n_points}")

## Transmitter Definition

In [ ]:
@dataclass
class Transmitter:
    tx_id: str
    lon: float
    lat: float
    htg: float
    f: float
    pol: int
    p: float
    hrg: float

tx = Transmitter(
    tx_id=CONFIG['TRANSMITTER']['tx_id'],
    lon=CONFIG['TRANSMITTER']['longitude'],
    lat=CONFIG['TRANSMITTER']['latitude'],
    htg=CONFIG['TRANSMITTER']['antenna_height_tx'],
    f=CONFIG['P1812']['frequency_ghz'],
    pol=CONFIG['P1812']['polarization'],
    p=CONFIG['P1812']['time_percentage'],
    hrg=CONFIG['TRANSMITTER']['antenna_height_rx'],
)
print(tx)

## Generate Receivers and Fetch Land Cover

In [ ]:
def generate_receivers_radial_multi(
    tx,
    distances_km: Iterable[Union[int, float]],
    azimuths_deg: Iterable[Union[int, float]],
    include_tx_point: bool = False,
):
    """Generate receivers on multiple rings around transmitter."""
    tx_gdf = gpd.GeoDataFrame(
        {"tx_id": [tx.tx_id]},
        geometry=[Point(tx.lon, tx.lat)],
        crs="EPSG:4326",
    )
    utm_crs = tx_gdf.estimate_utm_crs()
    tx_utm = tx_gdf.to_crs(utm_crs)
    tx_pt = tx_utm.geometry.iloc[0]

    rows = []
    rx_id = 1

    if include_tx_point:
        rows.append({
            "tx_id": tx.tx_id,
            "rx_id": 0,
            "distance_km": 0.0,
            "geometry": Point(tx.lon, tx.lat),
        })

    for d_km in distances_km:
        radius_m = float(d_km) * 1000.0
        for az in azimuths_deg:
            theta = math.radians(float(az))
            dx = radius_m * math.sin(theta)
            dy = radius_m * math.cos(theta)
            rx_utm = Point(tx_pt.x + dx, tx_pt.y + dy)
            rx_ll = gpd.GeoSeries([rx_utm], crs=utm_crs).to_crs("EPSG:4326").iloc[0]
            rows.append({
                "tx_id": tx.tx_id,
                "rx_id": rx_id,
                "distance_km": float(d_km),
                "azimuth_deg": float(az),
                "geometry": rx_ll,
            })
            rx_id += 1

    return gpd.GeoDataFrame(rows, geometry="geometry", crs="EPSG:4326")

# Generate receiver points
receivers = generate_receivers_radial_multi(tx, distances, azimuths, include_tx_point=True)
print(f"Generated {len(receivers)} receiver points")

In [ ]:
# Fetch and cache land cover from Sentinel Hub
client_id, client_secret = resolve_credentials(
    fallback_id=SH_CLIENT_ID,
    fallback_secret=SH_CLIENT_SECRET,
)

lat = CONFIG['TRANSMITTER']['latitude']
lon = CONFIG['TRANSMITTER']['longitude']
buffer_m = CONFIG['SENTINEL_HUB']['buffer_m']
chip_px = CONFIG['SENTINEL_HUB']['chip_px']
year = CONFIG['SENTINEL_HUB']['year']

out_tif = api_data_dir / f"lcm10_{lat}_{lon}_{year}_buf{buffer_m}m_{chip_px}px.tif"

# Only fetch if not cached
if not out_tif.exists():
    print(f"Fetching land cover data...")
    code, chip = landcover_at_point(
        client_id, client_secret,
        lat, lon,
        token_url=TOKEN_URL,
        process_url=PROCESS_URL,
        collection_id=COLLECTION_ID,
        year=year,
        buffer_m=buffer_m,
        chip_px=chip_px,
        save_path=str(out_tif)
    )
    print(f"Saved GeoTIFF: {out_tif}")
else:
    print(f"Using cached GeoTIFF: {out_tif}")

## Extract Profiles and Export

In [ ]:
rows = []
tif_path_str = str(out_tif)

for i, az in enumerate(azimuths):
    print(f"[{i+1}/{len(azimuths)}] Processing azimuth {az}°...", end=" ")

    try:
        # Generate profile using module function
        gdf = generate_profile_points(
            tx_lon, tx_lat,
            max_distance_km,
            n_points,
            azimuth_deg=az,
            tif_path=tif_path_str,
            lcm10_to_ct=CONFIG['LCM10_TO_CT'],
            ct_to_r=CONFIG['CT_TO_R'],
            zones_path=None,  # Optional: point to zones_map_BR.json if available
        )

        # Extract coordinates
        phi_t, lam_t = float(gdf.geometry.iloc[0].y), float(gdf.geometry.iloc[0].x)
        phi_r, lam_r = float(gdf.geometry.iloc[-1].y), float(gdf.geometry.iloc[-1].x)

        # Build row
        rows.append({
            "f": CONFIG['P1812']['frequency_ghz'],
            "p": CONFIG['P1812']['time_percentage'],
            "d": [float(round(v, 3)) for v in gdf["d"].tolist()],
            "h": [int(round(v)) if v else 0 for v in gdf["h"].tolist()],
            "R": [int(v) for v in gdf["R"].tolist()],
            "Ct": [int(v) for v in gdf["Ct"].tolist()],
            "zone": [int(v) for v in gdf["zone"].tolist()],
            "htg": CONFIG['TRANSMITTER']['antenna_height_tx'],
            "hrg": CONFIG['TRANSMITTER']['antenna_height_rx'],
            "pol": CONFIG['P1812']['polarization'],
            "phi_t": phi_t,
            "phi_r": phi_r,
            "lam_t": lam_t,
            "lam_r": lam_r,
        })
        print("✓")

    except Exception as e:
        print(f"✗ ({type(e).__name__}: {str(e)[:50]})")
        continue

# Save profiles
df_all = pd.DataFrame(rows)
output_path = profiles_dir / f"paths_oneTx_manyRx_{max_distance_km}km.csv"
df_all.to_csv(output_path, sep=";", index=False, decimal=".")

print(f"\n✓ Saved {len(df_all)} profiles to {output_path}")

## Summary

**Phase 2 Improvements**:
- Profile extraction functions now in `mst_gis.propagation.profile_extraction` module
- Reusable and testable functions
- Notebook simplified to 9 cells
- Ready for Phase 3 optimization (batch API calls)

Next: Run batch_processor.py to process profiles through P.1812-6 model.